In [27]:
def Tidclean(Tid):
    i = 0
    while Tid[i] == "0":
        i += 1
    return Tid[i:]

In [28]:
import csv
with open('data/legislators/legislators-current.csv', 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)
    bioguide_index = your_list[0].index("bioguide_id")
    thomas_index = your_list[0].index("thomas_id")
    party_index = your_list[0].index("party")
    legis_current = []
    for row in range(len(your_list)):
        person = your_list[row]
        if len(person[thomas_index]) == 5:
            thomas_id = Tidclean(person[thomas_index])
            legis_current.append([person[bioguide_index], thomas_id, person[party_index]])
        else:
            legis_current.append([person[bioguide_index], person[thomas_index], person[party_index]])
        
print(legis_current[:20])

[['bioguide_id', 'thomas_id', 'party'], ['B000944', '136', 'Democrat'], ['C000127', '172', 'Democrat'], ['C000141', '174', 'Democrat'], ['C000174', '179', 'Democrat'], ['C001070', '1828', 'Democrat'], ['F000062', '1332', 'Democrat'], ['K000367', '1826', 'Democrat'], ['M000639', '791', 'Democrat'], ['S000033', '1010', 'Independent'], ['S000770', '1531', 'Democrat'], ['T000464', '1829', 'Democrat'], ['W000802', '1823', 'Democrat'], ['B001261', '1881', 'Republican'], ['W000437', '1226', 'Republican'], ['A000360', '1695', 'Republican'], ['C001035', '1541', 'Republican'], ['C001056', '1692', 'Republican'], ['D000563', '326', 'Democrat'], ['E000285', '1542', 'Republican']]


In [29]:
import csv
with open('data/legislators/legislators-historical.csv', 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)
    bioguide_index = your_list[0].index("bioguide_id")
    thomas_index = your_list[0].index("thomas_id")
    party_index = your_list[0].index("party")
    legis_historical = []
    for row in range(len(your_list)):
        person = your_list[row]
        if len(person[thomas_index]) == 5:
            thomas_id = Tidclean(person[thomas_index])
            legis_historical.append([person[bioguide_index], thomas_id, person[party_index]])
        else:
            legis_historical.append([person[bioguide_index], person[thomas_index], person[party_index]])
        
print(legis_historical[:20])

[['bioguide_id', 'thomas_id', 'party'], ['B000226', '', 'Anti-Administration'], ['B000546', '', ''], ['B001086', '', ''], ['C000187', '', ''], ['C000538', '', ''], ['C000710', '', ''], ['D000013', '', 'Pro-Administration'], ['E000155', '', 'Pro-Administration'], ['F000100', '', 'Anti-Administration'], ['F000224', '', ''], ['G000012', '', ''], ['G000403', '', 'Anti-Administration'], ['G000500', '', ''], ['H000368', '', 'Pro-Administration'], ['H000995', '', ''], ['I000053', '', 'Pro-Administration'], ['J000182', '', 'Pro-Administration'], ['J000198', '', 'Pro-Administration'], ['L000201', '', 'Anti-Administration']]


party: The political party of the legislator. If the legislator changed parties, this is the most recent party held during the term and party_affiliations will be set. Values are typically "Democrat", "Independent", or "Republican". The value typically matches the political party of the legislator on the ballot in his or her last election, although for state affiliate parties such as "Democratic Farmer Labor" we will use the national party name ("Democrat") instead to keep the values of this field normalized.

In [49]:
legislators_ID = []
legislators_ID.extend(legis_current)
legislators_ID.extend(legis_historical[1:])

[['bioguide_id', 'thomas_id', 'party'],
 ['B000944', '136', 'Democrat'],
 ['C000127', '172', 'Democrat'],
 ['C000141', '174', 'Democrat'],
 ['C000174', '179', 'Democrat']]

In [58]:
import csv
with open('data/legislators/legislators_ID.csv', 'w') as outcsv:   
    writer = csv.writer(outcsv, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
    for item in legislators_ID:
        writer.writerow([item[0], item[1], item[2]])

In [69]:
with open('data/legislators/legislators_ID.csv', mode='r') as infile:
    reader = csv.reader(infile)
    dict_bioguide = {}
    dict_thomas = {}
    for person in reader:
        dict_bioguide[person[0]] = person[2]
        dict_thomas[person[1]] = person[2]

In [70]:
# first we do is for hr bills, congress nr 112
import json

lo_bills = []

# This is our module to get all the bill file paths.
from bill_paths import get_bill_file_paths 

# All the bills are organized alphabetically so instead of hr1, hr2, hr3 you'll get hr1, hr10, hr100 but the order is not relevant to us.
bill_paths = get_bill_file_paths(112)

#allyears = range(1, 117) #Current year is 116
#bill_paths = {}
#for year in allyears:
 #   bill_paths[year] = get_bill_file_paths(year) 
# if year 

for path in bill_paths:
    with open(path) as json_file:
        bill = json.load(json_file)
    # First check if the bill contains all the items we want for our project.
        if ("bill_id" and "official_title" and "subject_top_term" and "summary" and "sponsor") in bill.keys():
            if bill["summary"] != None:
                dictionary = {}
                dictionary["bill_id"] = bill["bill_id"]
    # We use the official title 
                dictionary["title"] = bill["official_title"] 
    # Most bills have many subjects so we decid to only look at the top term for clarity
                dictionary["subject"] = bill["subjects_top_term"]
                summary_ = bill["summary"]
                dictionary["summary"] = summary_["text"]
                sponsor = bill["sponsor"]
                # find party in csv file from ID
                if "thomas_id" in sponsor.keys():
                    thomas_id = sponsor["thomas_id"]
                    party = dict_thomas[Tidclean(thomas_id)]
                if "bioguide_id" in sponsor.keys():
                    party = dict_bioguide[sponsor["bioguide_id"]]
                dictionary["party"] = party
    
    # Appended to the list of bills 
        lo_bills.append(dictionary)
            
#per bill we create a dictionary containing the following data:
#Bill id, Title, Summary
# then, we create a list of all these dictionaries

In [72]:
import csv

with open('data/112_filtered.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter='|')
    filewriter.writerow(['bill_id', 'title', 'subject','summary', 'party'])
    for bill in lo_bills:
        filewriter.writerow([bill['bill_id'], bill['title'], bill['subject'], bill['summary'], bill['party']])